In [12]:
import os
import sys

# Add `super_resolution` package.
sys.path.insert(0, os.path.dirname(os.getcwd()))

import tensorflow as tf

In [13]:
from trainer import predict
from trainer import train
from trainer import model as straight_model
from trainer import angle_first_model
from preprocessing import parser
from preprocessing import input
from simulation import create_observation_spec


In [14]:
observation_spec_path='/Users/noah/Documents/CHU/super_resolution/data/simulation/circle_3_18/observation_spec.json'

predict_dataset_directory='/Users/noah/Documents/CHU/super_resolution/data/simulation/circle_3_18/predict'

model_dir='/Users/noah/Documents/CHU/super_resolution/experiments/3_26_straight'
model_type="STRAIGHT"


distribution_blur_sigma=1e-4
observation_blur_sigma=1e-4
distribution_downsample_size=(250, 250)
observation_downsample_size=(250, 250)
example_size=(501,501)

In [15]:
observation_spec = create_observation_spec.load_observation_spec(observation_spec_path)

predict_parse_fn = parser.Parser(
observation_spec=observation_spec,
reverse_rotation=True,
distribution_blur_sigma=distribution_blur_sigma,
observation_blur_sigma=observation_blur_sigma,
distribution_downsample_size=distribution_downsample_size,
observation_downsample_size=observation_downsample_size,
example_size=example_size,
).parse

if model_type==train._STRAIGHT:
    model=straight_model
elif model_type==train._ANGLE_FIRST:
    model=angle_first_model
else:
    raise ValueError('Not a valid model type. Got {}'.format(args.model_type))

estimator_fn = model.build_estimator
hparams = model.make_hparams()

hparams['observation_spec'] = observation_spec



In [ ]:
# Set up input
predict_dataset = input.input_fn(
      predict_dataset_directory,
      predict_parse_fn,
      1,
      1,
      1,
      1,
    )
iterator = predict_dataset.make_one_shot_iterator()
features, labels = iterator.get_next()

# Load `RunConfig`.
run_config = tf.estimator.RunConfig()
run_config = run_config.replace(model_dir=model_dir)

# Rebuild the model
predictions = model.model_fn(features, labels, tf.estimator.ModeKeys.EVAL, hparams).predictions

# Manually load the latest checkpoint
saver = tf.train.Saver()
with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(model_dir)
    saver.restore(sess, ckpt.model_checkpoint_path)

    # Loop through the batches and store predictions and labels
    prediction_values = []
    label_values = []
    for i in range(1):
        try:
            preds, lbls = sess.run([predictions, labels])
            prediction_values += preds
            label_values += lbls
        except tf.errors.OutOfRangeError:
            break


INFO:tensorflow:Restoring parameters from /Users/noah/Documents/CHU/super_resolution/experiments/3_26_straight/model.ckpt-4060


INFO:tensorflow:Restoring parameters from /Users/noah/Documents/CHU/super_resolution/experiments/3_26_straight/model.ckpt-4060
